In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 199 kB 8.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=511fc2a3b6a95a5b7ea3a9cf9f11431e4f9d60204d270d27fab0699429de4285
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps

In [3]:
spark = SparkSession \
        .builder \
        .master("local[1]")\
        .appName("Pokemon EDA") \
        .getOrCreate()

In [4]:
battle_df = ps.read_csv("/content/final_data.csv")
print(battle_df.info())

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `read_csv`, the default index is attached which can cause additional overhead.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


<class 'pyspark.pandas.frame.DataFrame'>
Int64Index: 49161 entries, 0 to 49160
Data columns (total 99 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Second_pokemon      49161 non-null  int32  
 1   First_pokemon       49161 non-null  int32  
 2   Winner              49161 non-null  int32  
 3   Legendary           49161 non-null  bool   
 4   name                49161 non-null  object 
 5   generation          49161 non-null  int32  
 6   status              49161 non-null  object 
 7   species             49161 non-null  object 
 8   type_number         49161 non-null  int32  
 9   type_1              49161 non-null  object 
 10  type_2              25455 non-null  object 
 11  height_m            49161 non-null  float64
 12  weight_kg           49161 non-null  float64
 13  abilities_number    49161 non-null  int32  
 14  ability_1           49161 non-null  object 
 15  ability_2           25580 non-null  object 
 16  a

WIN PERCENTAGES FOR POKEMON

In [5]:
wins = {}
battles = {}

temp = battle_df.get(['name', 'name_2', 'Winner'])

for i in temp.values:
  poke1 = i[0]
  poke2 = i[1]
  winner = i[2]
  if winner == 1:
    if poke1 not in wins.keys():
      wins[poke1] = 1
    else:
      wins[poke1] += 1
  else:
    if poke2 not in wins.keys():
      wins[poke2] = 1
    else:
      wins[poke2] += 1
  
  if poke1 not in battles.keys():
    battles[poke1] = 1
  else:
    battles[poke1] += 1
  
  if poke2 not in battles.keys():
    battles[poke2] = 1
  else:
    battles[poke2] += 1

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning: We recommend using `DataFrame.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))
/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [6]:
win_percentage = {}

for i in wins.keys():
  win_percentage[i] = wins[i] / battles[i]

In [7]:
import operator

high_win_percentage = dict(sorted(win_percentage.items(), key = operator.itemgetter(1), reverse = True))
low_win_percentage = dict(sorted(win_percentage.items(), key = operator.itemgetter(1), reverse = False))

In [8]:
ct = 0
pokemon = []
percentage = []

for i, j in high_win_percentage.items():
  if ct == 20:
    break

  pokemon.append(i)
  percentage.append(j*100)
  ct = ct + 1

Top 20 Pokemon with Highest Win Percentage

In [9]:
d = {'Pokemon' : pokemon, 'Win Percentage' : percentage}
high_percentage_df = ps.DataFrame(data = d, columns = ['Pokemon', 'Win Percentage'])
high_percentage_df.plot.bar(x = 'Pokemon', y = 'Win Percentage')

In [10]:
ct = 0
pokemon = []
percentage = []

for i, j in low_win_percentage.items():
  if ct == 20:
    break

  pokemon.append(i)
  percentage.append(j*100)
  ct = ct + 1

Bottom 20 Pokemon with Lowest Win Percentage

In [11]:
d = {'Pokemon' : pokemon, 'Win Percentage' : percentage}
low_percentage_df = ps.DataFrame(data = d, columns = ['Pokemon', 'Win Percentage'])
low_percentage_df.plot.bar(x = 'Pokemon', y = 'Win Percentage')

WIN PERCENTAGES FOR POKEMON TYPES

In [12]:
wins = {}
battles = {}

temp = battle_df.get(['type_1', 'type_1_2', 'Winner'])

for i in temp.values:
  type1 = i[0]
  type2 = i[1]
  winner = i[2]
  if winner == 1:
    if type1 not in wins.keys():
      wins[type1] = 1
    else:
      wins[type1] += 1
  else:
    if type2 not in wins.keys():
      wins[type2] = 1
    else:
      wins[type2] += 1
  
  if type1 not in battles.keys():
    battles[type1] = 1
  else:
    battles[type1] += 1
  
  if type2 not in battles.keys():
    battles[type2] = 1
  else:
    battles[type2] += 1

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



In [13]:
win_percentage = {}

for i in wins.keys():
  win_percentage[i] = wins[i] / battles[i]

In [14]:
import operator

win_percentage = dict(sorted(win_percentage.items(), key = operator.itemgetter(1), reverse = True))

In [15]:
pokemon = []
percentage = []

for i, j in win_percentage.items():
  pokemon.append(i)
  percentage.append(j*100)

In [16]:
d = {'Type' : pokemon, 'Win Percentage' : percentage}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

WIN PERCENTAGE AGAINST EACH TYPE

In [17]:
def get_win_percentage(poke_type):
  wins = {}
  battles = {}
  temp = battle_df.get(['type_1', 'type_1_2', 'Winner'])

  for i in temp.values:
    type1 = i[0]
    type2 = i[1]
    winner = i[2]

    if winner == 1 and type1 == poke_type:
      if type2 not in wins.keys():
        wins[type2] = 1
      else:
        wins[type2] += 1
    
    elif winner == 0 and type2 == poke_type:
      if type1 not in wins.keys():
        wins[type1] = 1
      else:
        wins[type1] += 1
    
    if type1 == poke_type:
      if type2 not in battles.keys():
        battles[type2] = 1
      else:
        battles[type2] += 1
    
    if type2 == poke_type: 
      if type1 not in battles.keys():
        battles[type1] = 1
      else:
        battles[type1] += 1
  
  win_percentage = {}

  for i in wins.keys():
    win_percentage[i] = (wins[i]/battles[i]) * 100

  win_percentage = dict(sorted(win_percentage.items(), key = operator.itemgetter(1), reverse = True))
  
  pokemon = []
  percentage = []

  for i, j in win_percentage.items():
    pokemon.append(i)
    percentage.append(j)

  return pokemon, percentage

Winning Percentage of Flying against all Types

In [18]:
import operator

poke, percent = get_win_percentage("Flying")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Rock against all Types

In [19]:
poke, percent = get_win_percentage("Rock")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Ground against all Types

In [20]:
poke, percent = get_win_percentage("Ground")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Electric against all Types

In [21]:
poke, percent = get_win_percentage("Electric")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Fire against all Types

In [22]:
poke, percent = get_win_percentage("Fire")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Dark against all Types

In [23]:
poke, percent = get_win_percentage("Dark")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Psychic against all Types

In [24]:
poke, percent = get_win_percentage("Psychic")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Dragon against all Types

In [25]:
poke, percent = get_win_percentage("Dragon")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Poison against all Types

In [26]:
poke, percent = get_win_percentage("Poison")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Ice against all Types

In [27]:
poke, percent = get_win_percentage("Ice")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Grass against all Types

In [28]:
poke, percent = get_win_percentage("Grass")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Steel against all Types

In [29]:
poke, percent = get_win_percentage("Steel")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Bug against all Types

In [30]:
poke, percent = get_win_percentage("Bug")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Normal against all Types

In [31]:
poke, percent = get_win_percentage("Normal")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Ghost against all Types

In [32]:
poke, percent = get_win_percentage("Ghost")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Water against all Types

In [33]:
poke, percent = get_win_percentage("Water")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Fairy against all Types

In [34]:
poke, percent = get_win_percentage("Fairy")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.



Winning Percentage of Fighting against all Types

In [35]:
poke, percent = get_win_percentage("Fighting")
d = {'Type' : poke, 'Win Percentage' : percent}
win_percentage_df = ps.DataFrame(data = d, columns = ['Type', 'Win Percentage'])
win_percentage_df.plot.bar(x = 'Type', y = 'Win Percentage')

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/generic.py:647: UserWarning:

We recommend using `DataFrame.to_numpy()` instead.

/usr/local/lib/python3.8/dist-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning:

`to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.

